In [ ]:
import data
import structures
import transformations as trans
import nibabel as nib
import torch as t
import os
import numpy as np
import open3d as o3d
import pytorch3d as p3

read nifti format and return image, affine and zooms as torch tensor, show slices (not in world coordinates)

In [ ]:
filename = "iFIND00472_iFIND2_201901111210_PHILIPSJ2LF9B9_101_PIH1HR_Survey_32SENSE.s3.nii.gz"
t_image, t_affine, zooms = data.nii_to_torch(filename)
data.show_stack(t_image)

Create stack object

In [ ]:
t_image_red = t_image[200:400,200:400,:]
data.show_stack(t_image_red)

In [ ]:
beta = 0.01
first_stack = structures.stack(t_image_red,t_affine, beta)

create target volume from desired geometry and the number of voxels in each direction

In [ ]:
corners = first_stack.corners()
corners

In [ ]:
geometry = corners
n_voxels = t.tensor([80,5,80])
target = structures.volume(geometry,n_voxels)

create slice and target volume

In [ ]:
target.register_stack(first_stack)

In [ ]:
nft_img = data.torch_to_nii(target.X, target.affine)
filename = 'first_nifti_target_jupyter'
data.save_nifti(nft_img,filename)

In [ ]:
[l_x,l_y,l_z] = geometry[:,0] - geometry[:,1]
x_lin, y_lin, z_lin = t.linspace(geometry[0,0],geometry[0,1],n_voxels[0]), t.linspace(geometry[1,0],geometry[1,1], n_voxels[1]), t.linspace(geometry[2,0],geometry[2,1], n_voxels[2])
x_grid, y_grid, z_grid = t.meshgrid(x_lin, y_lin, z_lin)
#coordinates
coordinates = t.stack((t.flatten(x_grid),t.flatten(y_grid),t.flatten(z_grid)),dim = 0)
#1 for "voxel structure" 0 for initial intensity value
add_on = t.tensor([[1],[0]]).repeat(1,coordinates.shape[1])
voxels = t.cat((coordinates,add_on),0)

In [ ]:
resolution = 9
geometry = t.tensor([[0,10],[-20,-5],[10,30]])
[l_x,l_y,l_z] = geometry[:,0] - geometry[:,1]

In [ ]:
n_voxels = t.ceil(t.abs(t.div(t.tensor([l_x,l_y,l_z]), resolution))).int()
x_lin, y_lin, z_lin = t.linspace(geometry[0,0],geometry[0,1],n_voxels[0]), t.linspace(geometry[1,0],geometry[1,1], n_voxels[1]), t.linspace(geometry[2,0],geometry[2,1], n_voxels[2])

In [ ]:
x_grid, y_grid, z_grid = t.meshgrid(x_lin, y_lin, z_lin)
#coordinates
coordinates = t.stack((t.flatten(x_grid),t.flatten(y_grid),t.flatten(z_grid)),dim = 0)
#1 for "voxel structure" 0 for initial intensity value
add_on = t.tensor([[1],[0]]).repeat(1,coordinates.shape[1])
voxels = t.cat((coordinates,add_on),0)

In [ ]:
voxels.shape